In [40]:
from bs4 import BeautifulSoup
import pandas as pd


In [41]:
html_path = 'Energetika/backUp/SunSetSunRise.html'
with open(html_path, 'r', encoding='ISO-8859-2') as f:
    html = f.read()
parsed_html = BeautifulSoup(html)

In [42]:
td_elements = parsed_html.find_all('td', class_='shadow')
for td in td_elements:
    data = []
    for td in td_elements:
        text = td.text.strip()
        parts = text.split('Východ:')
        if len(parts) > 1:
            day = parts[0].strip()
            times = parts[1].split('Západ:')
            sunrise = times[0].strip()
            sunset = times[1].strip()
            data.append([day, sunrise, sunset])

    df = pd.DataFrame(data, columns=['Day', 'SunRise','SunSet'])
    df['Day'] = df['Day'].apply(lambda x: x.split('.')[1] if '.' in x else x)

In [43]:
date_range = pd.date_range(start='2023-01-01', end='2023-12-31')
df['Date'] = date_range
df['SunRise'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['SunRise'], format='%Y-%m-%d %H:%M')
df['SunSet'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['SunSet'], format='%Y-%m-%d %H:%M')
df['Daylight'] = df['SunSet'] - df['SunRise'] # Compute the daylight duration
df['DaylightMinutes'] = df['Daylight'].dt.total_seconds() / 60 # Convert daylight duration to minutes


In [48]:
df = df[['Date', 'SunRise', 'SunSet', 'Daylight', 'DaylightMinutes']]
df.to_csv('Energetika/SunSetSunRise.csv', index=False)